# Check methzlation balance in annotation classes

---



## <span class="label label-success"> Analysis </span>



---

## <span class="label label-warning">NOTE</span>

- Very imbalanced

---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from Bio import SeqIO
from Bio.Seq import Seq

import os
import subprocess
import time

from IPython.core.display import Image 
from IPython.display import display

import re

In [2]:
workdir='/nagyvinyok/adat84/sotejedlik/ribli/methylation_code/explore_data'
subprocess.call(['mkdir',workdir])
os.chdir(workdir)
os.environ['TMPDIR']='/nagyvinyok/adat84/sotejedlik/ribli/tmp'

In [3]:
#sql exetcuter func
def run_sqlilte3(command,db,output=''):
    start=time.time()
    with open('tempf.sql','w') as tempf:
        tempf.write(command)
        
    if output != '':
        output=' > '+output
    
    try:
        print subprocess.check_output('/usr/bin/sqlite3 '+ db + ' < tempf.sql '+ output,
                                      shell=True, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError, e:
        print e.output
    
    subprocess.call(['rm','tempf.sql'])
    print 'It took',int(time.time()-start),'s'

In [4]:
pd.read_excel('GPL13534_450K_Manifest_header_Descriptions.xlsx')

,Column Name,Description
0,Index,Probe Index
1,TargetID,Identifies the probe name. Also used as a key ...
2,ProbeID_A,Illumina identifier for probe sequence A
3,ProbeID_B,Illumina identifier for probe sequence B
4,IlmnID,Unique CpG locus identifier from the Illumina ...
5,Name,Unique CpG locus identifier from the Illumina ...
6,AddressA_ID,Address of probe A
7,AlleleA_ProbeSeq,Sequence for probe A
8,AddressB_ID,Address of probe B
9,AlleleB_ProbeSeq,Sequence for probe B


In [13]:
run_sqlilte3('''
.separator "\t"

SELECT *
FROM ncbi_450_annot AS n
LIMIT 1;

''',db='../db/meth_db')

cg00035864	cg00035864	31729416	AAAACACTAACAATCTTATCCACATAAACCCTTAAATTTATCTCAAATTC			II			AATCCAAAGATGATGGAGGAGTGCCCGCTCATGATGTGAAGTACCTGCTCAGCTGGAAAC[CG]AATTTGAGATAAATTCAAGGGTCTATGTGGACAAGACTGCTAGTGTCTCTCTCTGGATTG	37	Y	8553009	AGACACTAGCAGTCTTGTCCACATAGACCCTTGAATTTATCTCAAATTCG	Y	8613009	F					TTTY18	NR_001550	TSS1500									

It took 0 s


### Regulatory feature group

In [6]:
run_sqlilte3('''
.separator "\t"
.load /home/ribli/tools/sqlite_math_ext/libsqlitefunctions

WITH med_table AS (
    SELECT probe AS probe,MEDIAN(beta) AS medb ,STDEV(beta) AS stdb
    FROM normal_h_meth
    GROUP BY probe)

SELECT n.Regulatory_Feature_Group,FLOOR(m.medb+0.5),COUNT(*)
FROM ncbi_450_annot AS n
INNER JOIN med_table AS m ON m.probe=n.IlmnID
WHERE ( m.medb > 0.8 OR m.medb < 0.2)  AND m.stdb <0.1
GROUP BY n.Regulatory_Feature_Group,FLOOR(m.medb+0.5);

''',db='../db/meth_db')

	0	35827
	1	182207
Gene_Associated	0	133
Gene_Associated	1	642
Gene_Associated_Cell_type_specific	0	47
Gene_Associated_Cell_type_specific	1	1412
NonGene_Associated	0	1056
NonGene_Associated	1	53
NonGene_Associated_Cell_type_specific	0	54
NonGene_Associated_Cell_type_specific	1	82
Promoter_Associated	0	78688
Promoter_Associated	1	1992
Promoter_Associated_Cell_type_specific	0	3421
Promoter_Associated_Cell_type_specific	1	747
Unclassified	0	17208
Unclassified	1	3294
Unclassified_Cell_type_specific	0	10663
Unclassified_Cell_type_specific	1	10133

It took 10 s


### UCSC CpG islands

In [14]:
run_sqlilte3('''
.separator "\t"
.load /home/ribli/tools/sqlite_math_ext/libsqlitefunctions

WITH med_table AS (
    SELECT probe AS probe,MEDIAN(beta) AS medb ,STDEV(beta) AS stdb
    FROM normal_h_meth
    GROUP BY probe)

SELECT n.Relation_to_UCSC_CpG_Island,FLOOR(m.medb+0.5),COUNT(*)
FROM ncbi_450_annot AS n
INNER JOIN med_table AS m ON m.probe=n.IlmnID
WHERE ( m.medb > 0.8 OR m.medb < 0.2)  AND m.stdb <0.1
GROUP BY n.Relation_to_UCSC_CpG_Island,FLOOR(m.medb+0.5);

''',db='../db/meth_db')

	0	11788
	1	108987
Island	0	97174
Island	1	20700
N_Shelf	0	780
N_Shelf	1	16695
N_Shore	0	17482
N_Shore	1	20779
S_Shelf	0	661
S_Shelf	1	15190
S_Shore	0	14741
S_Shore	1	15699

It took 8 s


### Strand

In [16]:
run_sqlilte3('''
.separator "\t"
.load /home/ribli/tools/sqlite_math_ext/libsqlitefunctions

WITH med_table AS (
    SELECT probe AS probe,MEDIAN(beta) AS medb ,STDEV(beta) AS stdb
    FROM normal_h_meth
    GROUP BY probe)

SELECT n.Strand,FLOOR(m.medb+0.5),COUNT(*)
FROM ncbi_450_annot AS n
INNER JOIN med_table AS m ON m.probe=n.IlmnID
WHERE ( m.medb > 0.8 OR m.medb < 0.2)  AND m.stdb <0.1
GROUP BY n.Strand,FLOOR(m.medb+0.5);

''',db='../db/meth_db')

F	0	72807
F	1	98908
R	0	69819
R	1	99142

It took 6 s


### Infinium Design Type

In [17]:
run_sqlilte3('''
.separator "\t"
.load /home/ribli/tools/sqlite_math_ext/libsqlitefunctions

WITH med_table AS (
    SELECT probe AS probe,MEDIAN(beta) AS medb ,STDEV(beta) AS stdb
    FROM normal_h_meth
    GROUP BY probe)

SELECT n.Infinium_Design_Type,FLOOR(m.medb+0.5),COUNT(*)
FROM ncbi_450_annot AS n
INNER JOIN med_table AS m ON m.probe=n.IlmnID
WHERE ( m.medb > 0.8 OR m.medb < 0.2)  AND m.stdb <0.1
GROUP BY n.Infinium_Design_Type,FLOOR(m.medb+0.5);

''',db='../db/meth_db')

I	0	71690
I	1	34483
II	0	70936
II	1	163567

It took 8 s


### Chromosome

In [18]:
run_sqlilte3('''
.separator "\t"
.load /home/ribli/tools/sqlite_math_ext/libsqlitefunctions

WITH med_table AS (
    SELECT probe AS probe,MEDIAN(beta) AS medb ,STDEV(beta) AS stdb
    FROM normal_h_meth
    GROUP BY probe)

SELECT n.Chromosome_36,FLOOR(m.medb+0.5),COUNT(*)
FROM ncbi_450_annot AS n
INNER JOIN med_table AS m ON m.probe=n.IlmnID
WHERE ( m.medb > 0.8 OR m.medb < 0.2)  AND m.stdb <0.1
GROUP BY n.Chromosome_36,FLOOR(m.medb+0.5);

''',db='../db/meth_db')

1	0	14960
1	1	17970
10	0	6583
10	1	10938
11	0	8703
11	1	11347
12	0	7823
12	1	9724
13	0	3028
13	1	5900
14	0	4672
14	1	6156
15	0	4735
15	1	6117
16	0	5678
16	1	9880
17	0	8832
17	1	10847
18	0	2125
18	1	2245
19	0	9460
19	1	8615
2	0	10065
2	1	14885
20	0	3796
20	1	3195
21	0	1179
21	1	1783
22	0	3061
22	1	2974
3	0	8121
3	1	10129
4	0	5811
4	1	8850
5	0	6968
5	1	9697
6	0	10921
6	1	15282
7	0	7406
7	1	14054
8	0	5228
8	1	9734
9	0	2819
9	1	3946
MULTI	1	28
X	0	647
X	1	3737
Y	0	5
Y	1	17

It took 8 s


### Random Loci

In [11]:
run_sqlilte3('''
.separator "\t"
.load /home/ribli/tools/sqlite_math_ext/libsqlitefunctions

WITH med_table AS (
    SELECT probe AS probe,MEDIAN(beta) AS medb ,STDEV(beta) AS stdb
    FROM normal_h_meth
    GROUP BY probe)

SELECT n.Random_Loci,FLOOR(m.medb+0.5),COUNT(*)
FROM ncbi_450_annot AS n
INNER JOIN med_table AS m ON m.probe=n.IlmnID
WHERE ( m.medb > 0.8 OR m.medb < 0.2)  AND m.stdb <0.1
GROUP BY n.Random_Loci,FLOOR(m.medb+0.5);

''',db='../db/meth_db')

	0	142296
	1	195368
TRUE	0	330
TRUE	1	2682

It took 9 s


### Methyl27_Loci

In [19]:
run_sqlilte3('''
.separator "\t"
.load /home/ribli/tools/sqlite_math_ext/libsqlitefunctions

WITH med_table AS (
    SELECT probe AS probe,MEDIAN(beta) AS medb ,STDEV(beta) AS stdb
    FROM normal_h_meth
    GROUP BY probe)

SELECT n.Methyl27_Loci,FLOOR(m.medb+0.5),COUNT(*)
FROM ncbi_450_annot AS n
INNER JOIN med_table AS m ON m.probe=n.IlmnID
WHERE ( m.medb > 0.8 OR m.medb < 0.2)  AND m.stdb <0.1
GROUP BY n.Methyl27_Loci,FLOOR(m.medb+0.5);

''',db='../db/meth_db')

	0	128873
	1	193368
TRUE	0	13753
TRUE	1	4682

It took 8 s


### Write relevant annotations

In [7]:
run_sqlilte3('''
.separator "\t"

SELECT IlmnID,Regulatory_Feature_Group,Relation_to_UCSC_CpG_Island,
    Strand,Infinium_Design_Type,Random_Loci,Methyl27_Loci
FROM ncbi_450_annot;

''',db='../db/meth_db',output='relevant_annotations.csv')


It took 2 s
